# Semesterprojekt Mock

## Anwendungsfälle:
- Planeten mocken bzw. die gesamte Galaxy (Sonnensystem)
- einzelne Methoden, um deren Unabhängigkeit zu gewährleisten
- Sonnensystem mocken, um im Falle einer Erweiterung auf noch nicht implementierte Methoden zugreifen zu können

## Mock der Klasse Galaxy

In [3]:
import json
from pathlib import Path
import time
from multiprocessing.shared_memory import SharedMemory

import numpy as np

_FPS = 60
_G = float()
_DISTANCE_SCALE = float()

# indexes for array containing planet data
_POS_SUN_IN_DATA = 0
_POS_X_DATA = 0
_POS_Y_DATA = 1
_POS_Z_DATA = 2
_MASS_DATA = 3
_VELOCITY_X_DATA = 4
_VELOCITY_Y_DATA = 5
_VELOCITY_Z_DATA = 6

# indexes for array containing rendered data
_POS_X_RENDERED = 0
_POS_Y_RENDERED = 1
_POS_Z_RENDERED = 2
_RADIUS_RENDERED = 3
_C_RED_RENDERED = 4
_C_GREEN_RENDERED = 5
_C_BLUE_RENDERED = 6
_C_ALPHA_RENDERED = 7

class Galaxy_Mock():
    def _calculate_acceleration(self, planet_index: int, planet_data: np.ndarray):
        """calculates the current acceleration for one planet regarding the forces between the planets

        Args:
            planetIndex: index in planet_data of the planet, whose acceleration should be calculated
            planet_data: 2d-array containing data for each planet (position, mass, velocity)

        Returns:
            np.array: vector of calculated new position
        """
        return np.array([1,0,0])

    def _calculate_new_position(self, planet_index: int, planet_data: np.ndarray, delta_t: float):
        """calculates new positon for one planet and sets new velocity according to calculated acceleration

        Args:
            planetIndex: index in planet_data of the planet, whose position should be calculated
            planet_data: 2d-array containing data for each planet (position, mass, velocity)
            delta_t: simulation step width

        Returns:
            np.array: vector of calculated new position
        """
        
        # skip the sun
        if planet_index == _POS_SUN_IN_DATA:
            return planet_data[planet_index][_POS_X_DATA:_POS_Z_DATA+1]
        
        curr_planet = planet_data[planet_index]  
        curr_pos = curr_planet[_POS_X_DATA:_POS_Z_DATA+1].copy()
        curr_velocity = curr_planet[_VELOCITY_X_DATA:_VELOCITY_Z_DATA+1].copy()
        
        accel = self._calculate_acceleration(planet_index, planet_data)
        newPos = curr_pos +  curr_velocity * delta_t + 1/2 * accel * delta_t**2
        # setting new velocity
        curr_planet[_VELOCITY_X_DATA:_VELOCITY_Z_DATA+1] = curr_velocity + accel * delta_t
        return newPos

    def _overwrite_rendered_positions(self, render_data: np.ndarray, planet_data: np.ndarray) -> None:
        """scales the new positions that should be renderd using the current positions from planet_data
            positions can only be rendered between -1 and 1, so they need to be scaled

        Args:
            render_data: information about the planets for rendering (position, radius, color)
            planet_data: 2d-array containing data for each planet (position, mass, velocity)
        """
        for planet_index, planet in enumerate(render_data[:]):
            newPosition = planet_data[planet_index]

            planet[_POS_X_RENDERED : _POS_Z_RENDERED+1] = newPosition[_POS_X_DATA : _POS_Z_DATA+1] * _DISTANCE_SCALE
            
    def _move_planets(self, planet_data: np.ndarray, delta_t: float) -> None:
        """calculates next position for each planet and after that saves the new positions in both given arrays

        Args:
            render_data: information about the planets for rendering (position, radius, color)
            planet_data: 2d-array containing data for each planet (position, mass, velocity)
            delta_t: simulation step width
        """
        new_positions = np.zeros((len(planet_data), 3), dtype=np.float32)
        for planet_index, planet in enumerate(planet_data[:]):
            newPos = self._calculate_new_position(planet_index, planet_data, delta_t)
            new_positions[planet_index] = newPos

        # setting new positions
        for pos_index, pos in enumerate(new_positions):
            planet_data[pos_index][_POS_X_DATA : _POS_Z_DATA+1] = pos.copy()
        
        time.sleep(1 / _FPS)

    def _load_config_file(self):
        cfg_path = f"{Path(__file__, '..').resolve()}/galaxy_cfg.json"
        with open (cfg_path) as cfg_file:
            global _cfg
            _cfg = json.load(cfg_file)

    def _initialize_planets(self, render_data: np.ndarray, planet_data: np.ndarray) -> None:
        """
        Get the position, mass, velocity out of the config file and put it into the 7-tupel for one planet
        Save the start position, radius and color values for each planet into the rendered array
        Set constant values like gravity coefficient and the distance scale
        Args:
            render_data: rendered data used for rendering containing the color values, positions and radius for each planet
            planet_array: Array of empty 7-tupels, each representing one planet
        """
        planet_data_cfg = _cfg["planets"]
        
        #format planet_data: (pos_x, pos_y, pos_z, mass, vel_x, vel_y, vel_z)
        for planet_index, planet in enumerate(planet_data[:]):
            curr_planet_cfg = planet_data_cfg[planet_index]
            
            # setting constant values
            global _G 
            _G = eval(_cfg["gravity_coefficient"])
            global _DISTANCE_SCALE 
            _DISTANCE_SCALE = eval(_cfg["distance_scale"])

            # filling planet data (not the rendered array)
            # mass 
            planet[_MASS_DATA] = eval(curr_planet_cfg["mass"])
            
            # velocity
            velocity = curr_planet_cfg["start_v"]
            planet[_VELOCITY_X_DATA:_VELOCITY_Z_DATA+1] = velocity
            
            # position
            curr_pos = eval(curr_planet_cfg["start_p"])
            planet[_POS_X_DATA:_POS_Z_DATA+1] = curr_pos
            
            # setting data for rendering into the rendered array
            # Radius
            # format rendered planet array: (pos_x, pos_y, pos_z, radius, c_red, c_green, c_blue, c_alpha)
            render_data[planet_index][_RADIUS_RENDERED] = curr_planet_cfg["radius"] * eval(_cfg["radius_scale"])
            
            # Color (RGB values)
            curr_rgb = curr_planet_cfg["c_rgb"]
            render_data[planet_index][_C_RED_RENDERED] = curr_rgb[0]
            render_data[planet_index][_C_GREEN_RENDERED] = curr_rgb[1]
            render_data[planet_index][_C_BLUE_RENDERED] = curr_rgb[2]
            render_data[planet_index][_C_ALPHA_RENDERED] = 1.0        
            
    def startup(self, shared_planets_name: str, shared_flags_name: str, delta_t: float):
        """
        Load values from config file
        Initialise and continuously update a position list.
        Initialize a list with all necessary data about the planets
        Results are sent through a pipe after each update step

        Args:
            shared_planets_name: Name of planets shared memory
            shared_flags_name: Name of flags shared memory
            delta_t: Simulation step width.
        """
        self._load_config_file()
        
        flags_shm = SharedMemory(shared_flags_name)
        shared_flags = flags_shm.buf
        planets_shm = SharedMemory(shared_planets_name)
        float32_nr = len(planets_shm.buf) // 4
        render_data = np.ndarray(
            shape=(float32_nr,), dtype=np.float32, buffer=planets_shm.buf
        )
        render_data = render_data.reshape(-1, 8)
        
        planet_data = np.zeros(shape=(len(_cfg["planets"]), 7))
        self._initialize_planets(render_data, planet_data)
        self._overwrite_rendered_positions(render_data, planet_data)
        time.sleep(2)
        while not shared_flags[1]:
            self._move_planets(planet_data, delta_t)
            self._overwrite_rendered_positions(render_data, planet_data)
            shared_flags[0] = 1
        for s in (flags_shm, planets_shm):
            s.close()


## Tests zur Klasse Galaxy

- Um die Berechnug der neuen Positionen der Planeten unabhängig testen zu können, wurde das Mock-Objekt der Klasse Galaxy verwendet. Innerhalb der gemockten Klasse gibt die Funktion, die die Beschleunigung der Planeten berechnet, einen konstanten Vektor zurück ([1,0,0]). 

In [14]:
""" Tests for module galaxy"""

import json
import numpy as np
import unittest
from unittest import mock


_SUN = 0
_MERCURY = 1
_VENUS = 2

class TestGalaxy(unittest.TestCase):
    
    def setUp(self):
        self._solarsystem = Galaxy_Mock()
        self.correct_cfg = json.loads('{"distance_scale": "0.98/(4539.24*10**(9))","radius_scale": "0.07/69911000","gravity_coefficient": "6.672*10**(-11)","delta_t": 30000,"planets": [{"name": "sun","c_rgb": [1.0, 0.835, 0],"mass": "1.98*10**(30)","radius": 6963400,"start_p": "[0,0,0]","start_v": [0, 0, 0]},{"name": "mercury","c_rgb": [0.557,0.529,0.467],"mass": "3.301*10**(23)","radius": 2439770,"start_p": "[46.001*10**(9),0,0]","start_v": [0,53078.4,25707.04241]},{"name": "venus","c_rgb": [0.835,0.733,0.494],"mass": "4.875*10**(24)","radius": 6051800,"start_p": "[107.473*10**(9),0,0]","start_v": [0,35260,0]}]}')
        _G = eval(self.correct_cfg["gravity_coefficient"])
        _DISTANCE_SCALE = eval(self.correct_cfg["distance_scale"])
        
        sun_data = self.correct_cfg["planets"][_SUN]
        mercury_data = self.correct_cfg["planets"][_MERCURY]
        venus_data = self.correct_cfg["planets"][_VENUS]
        self.correct_d_scale = eval(self.correct_cfg["distance_scale"])
        self.correct_r_scale = eval(self.correct_cfg["radius_scale"])
        
        # Set up the rendering array before overwritten positions
        self.correct_render_data = np.array([
            [0,0,0,
             sun_data["radius"]*self.correct_r_scale,
             sun_data["c_rgb"][0],sun_data["c_rgb"][1],sun_data["c_rgb"][2],1.0],

            [0,0,0,
             mercury_data["radius"]*self.correct_r_scale,
             mercury_data["c_rgb"][0],mercury_data["c_rgb"][1],mercury_data["c_rgb"][2],1.0],
            
            [0,0,0,
             venus_data["radius"]*self.correct_r_scale,
             venus_data["c_rgb"][0],venus_data["c_rgb"][1],venus_data["c_rgb"][2],1.0]])     
        
        # Set up the rendering array after overwritten positions
        self.correct_render_data_with_pos = np.array([
            [eval(sun_data["start_p"])[0]*self.correct_d_scale,
             eval(sun_data["start_p"])[1]*self.correct_d_scale,
             eval(sun_data["start_p"])[2]*self.correct_d_scale,
             sun_data["radius"]*self.correct_r_scale,
             sun_data["c_rgb"][0],sun_data["c_rgb"][1],sun_data["c_rgb"][2],1.0],

            [eval(mercury_data["start_p"])[0]*self.correct_d_scale,
             eval(mercury_data["start_p"])[1]*self.correct_d_scale,
             eval(mercury_data["start_p"])[2]*self.correct_d_scale,
             mercury_data["radius"]*self.correct_r_scale,
             mercury_data["c_rgb"][0],mercury_data["c_rgb"][1],mercury_data["c_rgb"][2],1.0],
            
            [eval(venus_data["start_p"])[0]*self.correct_d_scale,
             eval(venus_data["start_p"])[1]*self.correct_d_scale,
             eval(venus_data["start_p"])[2]*self.correct_d_scale,
             venus_data["radius"]*self.correct_r_scale,
             venus_data["c_rgb"][0],venus_data["c_rgb"][1],venus_data["c_rgb"][2],1.0]])     
        
        # Setup the planet array with realistic values for calculation
        self.correct_planet_data = np.array([
            [eval(sun_data["start_p"])[0],
             eval(sun_data["start_p"])[1],
             eval(sun_data["start_p"])[2],
             eval(sun_data["mass"]),
             sun_data["start_v"][0],sun_data["start_v"][1],sun_data["start_v"][2]],

            [eval(mercury_data["start_p"])[0],
             eval(mercury_data["start_p"])[1],
             eval(mercury_data["start_p"])[2],
             eval(mercury_data["mass"]),
             mercury_data["start_v"][0],mercury_data["start_v"][1],mercury_data["start_v"][2]],
            
            [eval(venus_data["start_p"])[0],
             eval(venus_data["start_p"])[1],
             eval(venus_data["start_p"])[2],
             eval(venus_data["mass"]),
             venus_data["start_v"][0],venus_data["start_v"][1],venus_data["start_v"][2]]]) 
        
          

    def test_calculate_acceleration(self) -> None:
        self.setUp()
        correct_mercury_acceleration = [1, 0, 0]
        epsilon = 5
        test_mercury_acceleration = self._solarsystem._calculate_acceleration(_MERCURY, self.correct_planet_data)
        np.testing.assert_array_almost_equal(test_mercury_acceleration, correct_mercury_acceleration, epsilon) 
        
    def test_calculate_new_position(self) -> None:
        self.setUp()
        correct_mercury_position = eval("[4.6451*10**(3), 1.592352*10**(2), 7.71211272*10]")
        epsilon = 5
        test_mercury_position = self._solarsystem._calculate_new_position(_MERCURY, self.correct_planet_data, self.correct_cfg["delta_t"])*10**(-7)
        np.testing.assert_array_almost_equal(test_mercury_position, correct_mercury_position, epsilon) 
                
    
if __name__ == "__main__":
    suite = unittest.TestLoader().loadTestsFromTestCase(TestGalaxy)
    unittest.TextTestRunner(verbosity=1).run(suite)
    



..
----------------------------------------------------------------------
Ran 2 tests in 0.007s

OK
